In [41]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable

In [19]:
NUM_AMMINO = 21
SEQ_LEN = 4

In [ ]:
TRANSLATE = 

In [2]:
data = pd.read_csv("task3_ks39mcp5/train.csv")

In [3]:
data

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
...,...,...
111995,GSME,0
111996,DLPT,0
111997,SGHC,0
111998,KIGT,0


In [16]:
def sequence(seq):
    vector = [ord(letter) for letter in seq]
    
    return vector

In [17]:
def bitmask(seq, alphabet=None):
    if alphabet==None:
        alphabet = ['R', 'H', 'K', 'D', 'E', 'S', 'T', 'N', 'Q', 'C', 'U', 'G', 'P', 'A', 'I', 'L', 'M', 'F', 'W', 'Y', 'V']
    
    vector = [[0 if char != letter else 1 for char in alphabet] for letter in seq]
    
    return vector

In [27]:
y_train = data['Active']

In [22]:
X_train = np.zeros((data.shape[0], SEQ_LEN, 1))
for i in range(data.shape[0]):
    X_train[i,:,0] = sequence(data.iloc[i, 0])

In [43]:
X_train = Variable(torch.Tensor(X_train))
y_train = Variable(torch.Tensor(y_train))

In [32]:
class lstm(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(lstm, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [34]:
lstm = lstm(1, 1, 3, 3, 4) #our lstm class 

In [37]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01) 

In [48]:
for epoch in range(1000):
      outputs = lstm.forward(X_train) #forward pass
      optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
      # obtain the loss function
      loss = criterion(outputs, y_train)
 
      loss.backward() #calculates the loss of the loss function
 
      optimizer.step() #improve from loss, i.e backprop
      if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 150528000000 bytes. Error code 12 (Cannot allocate memory)

In [46]:
X_train.shape

torch.Size([112000, 4, 1])